## imports

In [1]:
import os
from geopandas import read_file
from geojson import Feature, FeatureCollection, dump
import json
import simplekml
from pyproj import transform, Proj

data_partition = 'd:\\data\\resow'

hexgrid_dir = os.path.join(data_partition, 'hexgrid')
base_file_path = os.path.join(hexgrid_dir, 'hexgrid_base.geojson')
out_file_path = os.path.join(hexgrid_dir, 'hexgrid2.geojson')

base_grid = read_file(base_file_path)

crs = {
        "type": "name",
        "properties": {"name": base_grid.crs['init']}
      }

def WKT(hexgrid_df):
    
    west = hexgrid_df['west'].values[0]
    east = hexgrid_df['east'].values[0]
    north = hexgrid_df['north'].values[0]
    south = hexgrid_df['south'].values[0]
    
    print(f'POLYGON(({west} {north}, {west} {south} ,' + \
           f'{east} {south}, {east} {north}, {west} {north}))')

## run the code

In [ ]:
features = []
for index, row in base_grid.iterrows():
    features.append(Feature(geometry=row['geometry'],
                            properties={'id': index}))

feature_collection = FeatureCollection(features, crs=crs)

with open(out_file_path, 'w') as f:
   dump(feature_collection, f)

In [ ]:
for cell in [23, 63, 88, 128, 204, 258]:
    
    hexcell_file_path = os.path.join(hexgrid_dir, f'hexgrid_{str(cell)}.geojson')

    features = []
    for index, row in base_grid.iterrows():
        
        if index == cell:
            features.append(Feature(geometry=row['geometry'],
                            properties={'id': index,
                                        'west': row['left'],
                                        'east': row['right'],
                                        'south': row['bottom'],
                                        'north': row['top']}))
            
    feature_collection = FeatureCollection(features, crs=crs)

    with open(hexcell_file_path, 'w') as f:
       dump(feature_collection, f)
    
    WKT(read_file(hexcell_file_path))
    print()

In [10]:
with open(base_file_path) as f:
    data = json.load(f)


projectionIn = Proj(init='epsg:32627')

kml = simplekml.Kml()
for feature in data['features']:
    coord_tuples = []
    coord_list = (feature['geometry']['coordinates'][0][0])
    for coord in coord_list:
        long_w, lat_n = projectionIn(coord[0], coord[1], inverse=True)
#        long_w, lat_n = transform(projectionIn, projectionIn,
#                                  coord[0], coord[1])
        coord_tuples.append((long_w, lat_n))
    print(coord_tuples)
    name = str((int(feature['properties']['id'])))
    if feature['geometry']['type'] == 'MultiPolygon':
        kml.newpolygon(name=name,
                       description='test',
                       outerboundaryis=coord_tuples)
kml.save(base_file_path.replace('geojson', 'kml'))

[(-25.67352267671117, 8.082554490313361), (-25.60780714006235, 8.195105460028776), (-25.47710020934014, 8.194693047881003), (-25.41218095745455, 8.081648607544167), (-25.477965444876503, 7.969112102659837), (-25.60859650934347, 7.969559283687528), (-25.67352267671117, 8.082554490313361)]
[(-25.47710020934014, 8.194693047881003), (-25.411308921664336, 8.307246692439401), (-25.2805319315917, 8.306740085305906), (-25.215621268525073, 8.193646807441386), (-25.28147863438218, 8.081107975858295), (-25.41218095745455, 8.081648607544167), (-25.47710020934014, 8.194693047881003)]
[(-25.08284674453954, 8.644416391947123), (-25.01689277195678, 8.756982863713487), (-24.885906864216214, 8.75640688722355), (-24.820953772548556, 8.643234438988198), (-24.88697275157219, 8.530686042728233), (-25.017880507283365, 8.531291942902802), (-25.08284674453954, 8.644416391947123)]
[(-25.08593793011935, 7.967378522123661), (-25.0200168435719, 8.079902140288233), (-24.88925930287289, 8.079236293502989), (-24.8244